In [13]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
with np.load('./notMNIST.npz') as data :
    Data, Target = data ["images"], data["labels"]
    np.random.seed(521)
    randIndx = np.arange(len(Data))
    np.random.shuffle(randIndx)
    Data = Data[randIndx]/255.
    Target = Target[randIndx]
    trainData, trainTarget = Data[:15000], Target[:15000]
    validData, validTarget = Data[15000:16000], Target[15000:16000]
    testData, testTarget = Data[16000:], Target[16000:]

In [15]:
trainZeros=np.zeros((15000, 10))
trainZeros[np.arange(15000),trainTarget]=1
trainTarget = trainZeros
validZeros=np.zeros((1000, 10))
validZeros[np.arange(1000),validTarget]=1
validTarget = validZeros
testZeros=np.zeros((2724, 10))
testZeros[np.arange(2724),testTarget]=1
testTarget = testZeros

In [16]:
n_epochs = 1000
batch_size = 500
n_dim = 28*28

In [17]:
def grab_batches(trainData, trainTarget, batch_size):
    batch_indices = np.random.permutation(range(15000)).reshape(-1, batch_size)
    X_batches = trainData.reshape(-1, n_dim)[batch_indices]
    y_batches = trainTarget[batch_indices]
    batches = zip(X_batches, y_batches)
    return batches

In [18]:
X = tf.placeholder(tf.float32,[None,n_dim])
Y = tf.placeholder(tf.float32,[None,10])
lr = tf.placeholder(tf.float32)

In [19]:
def hidden_layer(X, hidden_units):
    x_dimension = X.shape[1].value
    initializer = tf.contrib.layers.xavier_initializer(uniform=False)
    hidden_weights = tf.Variable(initializer([x_dimension, hidden_units]), name='weights')
    hidden_biases = tf.Variable(tf.zeros(hidden_units), name='biases')
    
    return tf.add(tf.matmul(X, hidden_weights), hidden_biases)

In [20]:
y_ = hidden_layer(X,1000)
y_ = hidden_layer(tf.nn.relu(y_),10)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=y_))
#regularizer = tf.nn.l2_loss(W)
#loss = tf.reduce_mean(loss + weight_decay * regularizer)

prediction = tf.cast(tf.round(tf.argmax(y_,1)), tf.int8)
equality = tf.equal(prediction, tf.cast(tf.argmax(Y,1), tf.int8))
accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))

training_step = tf.train.AdamOptimizer(lr).minimize(loss)
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [21]:
learning_rates = 10**np.random.uniform(low=-5, high=-1, size=8)
valid_metrics = {'loss': [], 'accuracy': []}
train_metrics = {'loss': [], 'accuracy': []}
test_metrics = {'loss': [], 'accuracy': []}

In [22]:
best_epoch = {'loss': 0, 'accuracy': 0}
best_loss = float('inf')
best_accuracy = 0
with tf.Session() as sess:
    for learning_rate in learning_rates:
        sess.run(init)
        print("Learning Rate: {} \n".format(learning_rate))
        for epoch in range(1,n_epochs+1):
            batches = grab_batches(trainData, trainTarget, batch_size)
            for X_batch, y_batch in batches:
                sess.run(training_step, feed_dict={X: X_batch, Y: y_batch, lr: learning_rate})

            train_loss, train_accuracy = sess.run([loss, accuracy], {X: trainData.reshape(-1,n_dim), Y: trainTarget})
            valid_loss, valid_accuracy = sess.run([loss, accuracy], {X: validData.reshape(-1,n_dim), Y: validTarget})
            test_loss, test_accuracy = sess.run([loss, accuracy], {X: testData.reshape(-1,n_dim), Y: testTarget})
            
            print("Epoch: {}, Loss: {}, Accuracy: {}".format(epoch, train_loss, train_accuracy))
            
            if valid_loss < best_loss:
                best_epoch['loss'] = epoch
                best_loss = valid_loss
            
            if valid_accuracy > best_accuracy:
                best_epoch['accuracy'] = epoch
                best_accuracy = valid_accyr
            
            if epoch%5 == 0:
                print("Best Epoch: {}, Best Loss: {}, Best Accuracy: {} \n".format(best_epoch, best_loss, best_accuracy))
                
            train_metrics['loss'].append(train_loss)
            train_metrics['accuracy'].append(train_accuracy)
            valid_metrics['loss'].append(valid_loss)
            valid_metrics['accuracy'].append(valid_accuracy)
            test_metrics['loss'].append(test_loss)
            test_metrics['accuracy'].append(test_accuracy)


Learning Rate: 0.002729584918752247 

Epoch: 1, Loss: 0.39875808358192444, Accuracy: 0.8880000114440918


NameError: name 'valid_accyract' is not defined